In [133]:
from sklearn.datasets import fetch_covtype
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# 1. 데이터 
datasets = (fetch_covtype())

x = pd.DataFrame(datasets["data"])
y = pd.DataFrame(datasets["target"])


In [134]:
print(x.shape,y.shape) 

print(np.unique(y)) # [0 1 2 3 4 5 6 7 8 9]
# 불균형확인
print(np.unique(y,return_counts=True)) # [0 1 2]



(581012, 54) (581012, 1)
[1 2 3 4 5 6 7]
(array([1, 2, 3, 4, 5, 6, 7]), array([211840, 283301,  35754,   2747,   9493,  17367,  20510],
      dtype=int64))


In [135]:
# print(x.info())
# print(x.describe())
# 데이터 분리
x_train, x_test, y_train,y_test = train_test_split(
                                                    x,
                                                    y,
                                                    train_size=0.8,
                                                    shuffle = True,
                                                    #  stratify 는 데이터 불균형을 해결해줌
                                                    stratify=y,
                                                    random_state=21
                                                   )
# print(x.columns)
# y 를 원 핫 인코딩 변환
y_train = tf.one_hot(y_train[0],depth =8)
# y 의 0 번 칼럼을 드랍
y_train = pd.DataFrame(y_train).drop(columns=[0])

y_test = tf.one_hot(y_test[0],depth= 8)
y_test = pd.DataFrame(y_test).drop(columns=[0])


print(y_train[:10])
print("y_train shape : ",y_train.shape)
print("y_test shape : ",y_test.shape)

     1    2    3    4    5    6    7
0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
1  1.0  0.0  0.0  0.0  0.0  0.0  0.0
2  1.0  0.0  0.0  0.0  0.0  0.0  0.0
3  0.0  1.0  0.0  0.0  0.0  0.0  0.0
4  1.0  0.0  0.0  0.0  0.0  0.0  0.0
5  0.0  1.0  0.0  0.0  0.0  0.0  0.0
6  0.0  1.0  0.0  0.0  0.0  0.0  0.0
7  0.0  1.0  0.0  0.0  0.0  0.0  0.0
8  1.0  0.0  0.0  0.0  0.0  0.0  0.0
9  1.0  0.0  0.0  0.0  0.0  0.0  0.0
y_train shape :  (464809, 7)
y_test shape :  (116203, 7)


In [136]:
model = Sequential([
    Dense(128,activation="relu",input_shape=(54,)),
    Dense(128,activation="relu") ,
    Dense(128,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(64,activation="relu") ,
    Dense(32,activation="relu") ,
    Dense(32,activation="relu") ,
    # 다중분류모델의 활성화 함수는 softmax 입니다
    Dense(7,activation="softmax")
])

In [137]:
# 이진분류는 마지막 활성함수는 sigmoid + loss 는 바이너리 크로스 엔트로피 
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
monitor='val_loss',
patience=10, 
verbose=1, 
mode='min')

# 훈련값이 int 형이기 때문에 sparse 를 사용합니다.
model.compile(loss="categorical_crossentropy"
              ,optimizer="adam"
              ,metrics=["accuracy"]
              )

hist = model.fit(x_train, y_train, epochs=10, 
                 validation_split=0.2,batch_size=256,
                 callbacks = [early_stopping])

# metrics 에 accuracy 사용가능


1453/1453 [==============================] - 6s 4ms/step - loss: 1.0777 - accuracy: 0.6121 - val_loss: 0.7242 - val_accuracy: 0.6872


In [138]:
loss,accuracy = model.evaluate(x_test,y_test)

print("loss:",loss,"\nacc :" ,accuracy)


3632/3632 [==============================] - 7s 2ms/step - loss: 0.7239 - accuracy: 0.6854
loss: 0.7238940000534058 
acc : 0.6853523850440979


In [139]:
from sklearn.metrics import accuracy_score

y_predict = model.predict(x_test)
print(y_predict)
y_predict = np.argmax(y_predict,axis=1)


[[8.0507183e-01 1.9293350e-01 5.7548364e-06 ... 5.0379842e-04
  1.4214102e-05 1.4708845e-03]
 [2.0194152e-01 7.6928645e-01 5.0954348e-03 ... 1.8013373e-02
  2.5316530e-03 3.1247435e-03]
 [2.1026553e-01 7.4448562e-01 1.3017231e-03 ... 3.0274384e-02
  1.3509805e-02 1.6201372e-04]
 ...
 [9.2255250e-02 8.6108822e-01 2.9099307e-03 ... 2.5130814e-02
  1.8378297e-02 2.3421628e-04]
 [1.3445111e-01 7.8506619e-01 4.6543046e-03 ... 6.7688033e-02
  7.9768309e-03 1.6194244e-04]
 [6.9962479e-02 8.9452869e-01 1.5909223e-02 ... 1.0438898e-02
  7.4643753e-03 1.6785437e-03]]


In [140]:
print(y_predict)

y_predict = tf.one_hot(y_predict,depth =7)


[0 1 1 ... 1 1 1]


In [141]:
# 데이터 출력

print("예측데이터 : ",y_predict[:20])
print("실제데이터 : ",y_test[:20])

print
print("========================================")
acc = accuracy_score(y_test,y_predict)
print(acc)


예측데이터 :  tf.Tensor(
[[1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]], shape=(20, 7), dtype=float32)
실제데이터 :        1    2    3    4    5    6    7
0   1.0  0.0  0.0  0.0  0.0  0.0  0.0
1   0.0  1.0  0.0  0.0  0.0  0.0  0.0
2   1.0  0.0  0.0  0.0  0.0  0.0  0.0
3   1.0  0.0  0.0  0.0  0.0  0.0  0.0
4   0.0  1.0  0.0  0.0  0.0  0.0  0.0
5   1.0  0.0  0.0  0.0  0.0  0.0  0.0
6   1.0  0.0  0.0  0.0  0.0  0.0  0.0
7   1.0  0.0  0.0  0.0  0.0  0.0  0.0
8   0.0  1.0  0.0  0.0  0.0  0.0  0.0
9   1.0  0.0  0.0  0.0  0.0  0.0  0.0
10  0.0  1.0  0.0  0.0  0.0  0.0  0.0
11 